### IMPORTS

In [1]:
import pandas as pd
import numpy as np
import pandas_gbq
import datetime
from dateutil.relativedelta import relativedelta
from Roster import *
from gspread_pandas import Spread, conf

### CREDENCIALES

In [2]:
cred = conf.get_config('C:\\Users\\micaela.fuchs\\Anaconda', 'PedidosYa-6e661fd93faf.json')

### CONSTANTES

In [3]:
# Fechas
today = datetime.date.today()
if today.day == 1:
    tm = str(today - relativedelta(days=1))
    lm = str((today - relativedelta(months=2)).replace(day=1))
else:
    tm = str(today + relativedelta(months=1) - relativedelta(days=(today + relativedelta(months=1)).day))
    lm = str((today - relativedelta(months=1)).replace(day=1))

### QUERIES

In [4]:
# 3.6GB
q_orders = '''SELECT SUBSTR(CAST(o.registered_date AS STRING),1,7) AS Month,
       c.city_name AS City,
       a.area_name AS Area,
       o.business_type.business_type_name AS Business,
       COUNT(DISTINCT(o.order_id)) AS Total,
       COUNT(DISTINCT(CASE WHEN o.order_status = 'CONFIRMED' THEN o.order_id ELSE NULL END)) AS Confirmed,
       COUNT(DISTINCT(CASE WHEN (o.order_status = 'CONFIRMED'  AND o.with_logistics = TRUE) THEN o.order_id ELSE NULL END)) AS Logistic,
       COUNT(DISTINCT(CASE WHEN (o.order_status = 'CONFIRMED' AND o.has_voucher_discount = 1) THEN o.order_id ELSE NULL END)) AS Vouchers_Orders,
       COUNT(DISTINCT(CASE WHEN (o.order_status = 'CONFIRMED' AND o.is_online_payment = 1) THEN o.order_id ELSE NULL END)) AS OP_Orders,
       COUNT(DISTINCT(CASE WHEN (o.order_status = 'REJECTED' AND o.fail_rate_owner = 'Restaurant') THEN o.order_id ELSE NULL END)) AS Rejected_Partner,
       COUNT(DISTINCT(CASE WHEN (o.order_status = 'REJECTED' AND o.fail_rate_owner = 'PedidosYa') THEN o.order_id ELSE NULL END)) AS Rejected_PEYA,
       COUNT(DISTINCT(CASE WHEN o.order_status = 'REJECTED' THEN o.order_id ELSE NULL END)) AS Rejected,
       COUNT(DISTINCT(CASE WHEN (o.order_status = 'PENDING' AND o.fail_rate_owner = 'Restaurant') THEN o.order_id ELSE NULL END)) AS Pending_Partner,
       COUNT(DISTINCT(CASE WHEN (o.order_status = 'PENDING' AND o.fail_rate_owner = 'PedidosYa') THEN o.order_id ELSE NULL END)) AS Pending_PEYA,
       COUNT(DISTINCT(CASE WHEN o.order_status = 'PENDING' THEN o.order_id ELSE NULL END)) AS Pending,
       COUNT(DISTINCT(CASE WHEN (o.order_status = 'CANCELLED' AND o.fail_rate_owner = 'Restaurant') THEN o.order_id ELSE NULL END)) AS Cancelled_Partner,
       COUNT(DISTINCT(CASE WHEN (o.order_status = 'CANCELLED' AND o.fail_rate_owner = 'PedidosYa') THEN o.order_id ELSE NULL END)) AS Cancelled_PEYA,
       COUNT(DISTINCT(CASE WHEN o.order_status = 'CANCELLED' THEN o.order_id ELSE NULL END)) AS Cancelled,
       ROUND(SUM(CASE WHEN o.order_status = 'CONFIRMED' AND o.with_logistics THEN (o.total_amount + o.shipping_amount) ELSE NULL END),3) AS GMV_Log,
       ROUND(SUM(CASE WHEN o.order_status = 'CONFIRMED' AND o.with_logistics = FALSE THEN (o.total_amount + o.shipping_amount) ELSE NULL END),3) AS GMV_Mktp,
       ROUND(SUM(CASE WHEN o.order_status = 'CONFIRMED' AND o.with_logistics THEN o.commission_amount ELSE NULL END),3) AS Revenue_Log,
       ROUND(SUM(CASE WHEN o.order_status = 'CONFIRMED' AND o.with_logistics = FALSE THEN o.commission_amount ELSE NULL END),3) AS Revenue_Mktp,
       ROUND(SUM(CASE WHEN o.order_status = 'CONFIRMED' AND o.with_logistics THEN o.shipping_amount_no_discount ELSE NULL END),3) AS DF_Log,
       ROUND(SUM(CASE WHEN o.order_status = 'CONFIRMED' AND o.with_logistics = FALSE THEN o.shipping_amount_no_discount ELSE NULL END),3) AS DF_Mktp,
       COUNT(DISTINCT(CASE WHEN o.order_status = 'CONFIRMED' AND o.with_logistics THEN o.order_id ELSE NULL END)) AS Log_Orders_Economics,
       COUNT(DISTINCT(CASE WHEN o.order_status = 'CONFIRMED' AND o.with_logistics = FALSE THEN o.order_id ELSE NULL END)) AS Mktp_Orders_Economics,
       ROUND(SUM(CASE WHEN o.order_status = 'CONFIRMED' AND o.is_online_payment = 1 THEN (o.total_amount + o.shipping_amount) ELSE NULL END),3) AS GMV_OP,
       ROUND(SUM(CASE WHEN o.order_status = 'CONFIRMED' AND o.is_online_payment = 1 THEN o.commission_amount ELSE NULL END),3) AS Revenue_OP,
       SUM(CASE WHEN o.order_status = 'CONFIRMED' AND o.with_logistics THEN o.total_amount + o.discount_paid_by_company + o.shipping_amount - o.shipping_amount_no_discount ELSE NULL END) AS Log_Income,
       SUM(CASE WHEN o.order_status = 'CONFIRMED' AND o.with_logistics = FALSE THEN o.total_amount + o.shipping_amount + o.discount_paid_by_company ELSE NULL END) AS Mktp_Income,
       COUNT(DISTINCT o.user.id) AS Total_Users,
       COUNT(DISTINCT CASE WHEN o.order_status = 'CONFIRMED' THEN o.user.id ELSE NULL END) AS Active_Users
FROM `peya-bi-tools-pro.il_core.fact_orders` as o
LEFT JOIN `peya-bi-tools-pro.il_core.dim_area` AS a ON o.address.area.id = a.area_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` AS c ON o.city.city_id = c.city_id
WHERE o.country_id = 3 
      AND o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
GROUP BY 1,2,3,4'''.format(lm,tm)

# 55MB
q_online = '''SELECT SUBSTR(CAST(DATE(pm.full_date) AS STRING),1,7) AS Month,
       p.city.name AS City,
       a.area_name AS Area,
       p.business_type.business_type_name AS Business,
       COUNT(DISTINCT(CASE WHEN pm.is_online THEN p.partner_id ELSE NULL END)) AS Online,
       COUNT(DISTINCT(CASE WHEN pm.is_online AND pm.is_logistic THEN p.partner_id ELSE NULL END)) AS OD,
       COUNT(DISTINCT(CASE WHEN pm.is_online AND pm.has_online_payment THEN p.partner_id ELSE NULL END)) AS OP,
       COUNT(DISTINCT(CASE WHEN pm.is_online AND pm.accepts_vouchers THEN p.partner_id ELSE NULL END)) AS Partners_Vouchers,
       SUM(CASE WHEN pm.has_confirmed_orders THEN pm.total_gold_vip + pm.total_featured_product + pm.total_gold_vip_category + pm.total_vip ELSE 0 END) AS NCR,
       SUM(CASE WHEN pm.has_confirmed_orders THEN pm.joker_fee ELSE 0 END) AS Joker
FROM `peya-bi-tools-pro.il_core.fact_partners_monthly` AS pm
LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` AS p ON pm.restaurant_id = p.partner_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_area` AS a ON p.address.area_id = a.area_id
WHERE DATE(pm.full_date) BETWEEN DATE('{0}') AND DATE('{1}')
      AND p.country_id = 3
GROUP BY 3,4,2,1'''.format(lm,tm)

# 430MB
q_sesiones = '''SELECT SUBSTR(CAST(sbr.date AS STRING),1,7) AS Month,
       p.city.name AS City,
       a.area_name AS Area,
       p.business_type.business_type_name AS Business,
       SUM(sbr.sessions) AS Sessions
FROM `peya-bi-tools-pro.il_sessions.fact_sessions_by_restaurant` AS sbr
LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` as p ON sbr.restaurant_id = p.partner_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_area` AS a ON p.address.area_id = a.area_id
WHERE p.country_id = 3 
      AND sbr.date BETWEEN DATE('{0}') AND DATE('{1}')
GROUP BY 2,3,4,1'''.format(lm,tm)

# 2.2GB
q_op = '''SELECT SUBSTR(CAST(o.registered_date AS STRING),1,7) AS Month,
       c.city_name AS City,
       a.area_name AS Area,
       o.business_type.business_type_name AS Business,
       IFNULL(COUNT(DISTINCT CASE WHEN lo.is_vendor_late_10 = 1 THEN o.order_id ELSE NULL END),0) AS VL10_Numerador,
       IFNULL(COUNT(DISTINCT CASE WHEN lo.is_vendor_late_nn = 1 THEN o.order_id ELSE NULL END),0) AS VL_Denominador
FROM `peya-bi-tools-pro.il_logistics.fact_logistic_orders` AS lo
LEFT JOIN `peya-bi-tools-pro.il_core.fact_orders` AS o ON lo.peya_order_id = o.order_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_area` AS a ON o.address.area.id = a.area_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` AS c ON o.city.city_id = c.city_id
WHERE lo.created_date_local BETWEEN DATE('{0}') AND DATE('{1}')
      AND o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
      AND o.country_id = 3
GROUP BY 1,2,3,4'''.format(lm,tm)

# 2.1GB
q_acq = '''SELECT SUBSTR(CAST(o.registered_date AS STRING),1,7) AS Month,
       c.city_name AS City,
       a.area_name AS Area,
       o.business_type.business_type_name AS Business,
       COUNT(DISTINCT CASE WHEN obc.nro_order_confirmed = 1 THEN obc.user_id ELSE NULL END) AS Acquisitions
FROM `peya-bi-tools-pro.il_core.fact_peya_orders_by_customers` AS obc
LEFT JOIN `peya-bi-tools-pro.il_core.fact_orders` AS o ON obc.order_id = o.order_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_area` AS a ON o.address.area.id = a.area_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` AS c ON o.city.city_id = c.city_id
WHERE o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
      AND DATE(obc.registered_date) BETWEEN DATE('{0}') AND DATE('{1}')
GROUP BY 3,4,2,1'''.format(lm,tm)

In [5]:
# Descargo la data
hue_online = pd.io.gbq.read_gbq(q_online, project_id='peya-argentina', dialect='standard')

Downloading: 100%|██████████| 7650/7650 [00:01<00:00, 5976.52rows/s]


In [6]:
hue_orders = pd.io.gbq.read_gbq(q_orders, project_id='peya-argentina', dialect='standard')

Downloading: 100%|██████████| 21350/21350 [00:09<00:00, 2253.80rows/s]


In [7]:
hue_acq = pd.io.gbq.read_gbq(q_acq, project_id='peya-argentina', dialect='standard')

Downloading: 100%|██████████| 57338/57338 [00:05<00:00, 10644.67rows/s]


In [8]:
hue_sesiones = pd.io.gbq.read_gbq(q_sesiones, project_id='peya-argentina', dialect='standard')

Downloading: 100%|██████████| 6074/6074 [00:01<00:00, 5683.40rows/s]


In [9]:
hue_op = pd.io.gbq.read_gbq(q_op, project_id='peya-argentina', dialect='standard')

Downloading: 100%|██████████| 18817/18817 [00:02<00:00, 8512.95rows/s]


In [10]:
# Copia las bases
orders = hue_orders.copy()
online = hue_online.copy()
sesiones = hue_sesiones.copy()
op = hue_op.copy()
acq = hue_acq.copy()

### TRABAJO

In [11]:
# Doy el mismo formato a todos los Months
orders['Month'] = pd.to_datetime(orders['Month'], format='%Y-%m').dt.strftime('%Y-%m')
online['Month'] = pd.to_datetime(online['Month'], format='%Y-%m').dt.strftime('%Y-%m')
sesiones['Month'] = pd.to_datetime(sesiones['Month'], format='%Y-%m').dt.strftime('%Y-%m')
op['Month'] = pd.to_datetime(op['Month'], format='%Y-%m').dt.strftime('%Y-%m')
acq['Month'] = pd.to_datetime(acq['Month'], format='%Y-%m').dt.strftime('%Y-%m')

In [12]:
# Hago el merge
cols = ['Month','City','Area','Business']
cities = orders.merge(online,on=cols,how='outer')
cities = cities.merge(sesiones,on=cols,how='outer')
cities = cities.merge(op,on=cols,how='outer')
cities = cities.merge(acq,on=cols,how='outer')
cities = cities.replace([np.inf, -np.inf, np.nan], 0).copy()

In [13]:
# Coloco los Feudos y Reinos
cities['Feudo Asignado'] = cities.apply(asignado,axis=1)
cities = cities[cities['Feudo Asignado'] == 'Si'].copy()
cities['Feudo'] = cities.apply(feudos,axis=1)
cities['Reino'] = cities.apply(reinos,axis=1)
cities['Vertical'] = cities['Business'].apply(lambda x: x if x == 'Restaurant' else 'NV')

In [14]:
# Doy formato necesario a las columnas
cols = [i for i in cities.columns if i not in ['Month','City','Area','Business','Feudo','Reino','Vertical','Feudo Asignado']]
cities[cols] = cities[cols].astype(float)

In [15]:
# Formo las columnas faltantes
cities['GMV'] = cities['GMV_Log'] + cities['GMV_Mktp']
cities['Revenue'] = cities['Revenue_Log'] + cities['Revenue_Mktp']
cities['Total_Rejected_Partner'] = cities['Rejected_Partner'] + cities['Pending_Partner'] + cities['Cancelled_Partner']
cities['Total_Rejected_PEYA'] = cities['Rejected_PEYA'] + cities['Pending_PEYA'] + cities['Cancelled_PEYA']
cities['Total_Rejected'] = cities['Rejected'] + cities['Pending'] + cities['Cancelled']
cities = cities.replace([np.inf, -np.inf, np.nan], 0).copy()

In [16]:
# Ordeno las columnas
cols = ['Month','City','Area','Business','Confirmed','Logistic','Vouchers_Orders','OP_Orders','Online','OD','OP','Partners_Vouchers',
        'Sessions','Rejected_Partner','Rejected_PEYA','Rejected','Pending_Partner','Pending_PEYA','Pending','Cancelled_Partner',
        'Cancelled_PEYA','Cancelled','Total','GMV_Log','GMV_Mktp','Revenue_Log','Revenue_Mktp','DF_Log','DF_Mktp','Log_Orders_Economics',
        'Mktp_Orders_Economics','GMV_OP','Revenue_OP','Log_Income','Mktp_Income','Total_Users','Active_Users','Acquisitions',
        'VL10_Numerador','VL_Denominador','NCR','Joker','Feudo','Reino','Vertical','GMV','Revenue','Total_Rejected_Partner',
        'Total_Rejected_PEYA','Total_Rejected']
cities = cities[cols].copy()

### CARGA

In [19]:
# Carga Cities 2.0
sheet_id = '1aiwWbTP2l_2aVNop5vGCdea9SYvlRCX50vzKQ9yicQI'
wks_name = 'Crudo'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(cities, index=False, sheet=wks_name, replace=True)

ReadTimeout: HTTPSConnectionPool(host='sheets.googleapis.com', port=443): Read timed out. (read timeout=120)